In [731]:
import pandas as pd
import numpy as np
import re

- "Lithium source", "Voltage range min", "Voltage range max", "Temperature" 변수 채점 자동화 처리 해야함!
- relevance rag 44번 논문 샘플명이 변형되어서 나옴
- 

In [732]:
hyper_method_name = "multiagent-rag_9"

In [733]:
gt = pd.read_csv("../output/csv/GT.csv", encoding="cp949")
answer = pd.read_csv(f"../output/csv/{hyper_method_name}.csv", encoding="cp949") ## .drop(columns="Additional treatment")

In [734]:
gt = gt.sort_values(by=['Paper ID', 'Sample']).reset_index(drop=True)
answer = answer.sort_values(by=['Paper ID', 'Sample']).reset_index(drop=True)

In [735]:
gt["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)[0] 
answer["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)[0]

nan

In [736]:
def text_processor(df): 
    ## `Voltage range` 변수 처리
    df["Voltage range"] = df["Voltage range"].apply(
        lambda x: x.replace("V", "").replace("v", "").replace("?", "-").replace("--", "-") if x is not np.NaN else None
    )
    
    df["Voltage range min"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[0]) if isinstance(x, str) and '-' in x else None
    )

    df["Voltage range max"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[1]) if isinstance(x, str) and '-' in x else None
    )

    df = df.drop(columns=["Voltage range"])


    ## `Temperature` 변수 처리
    df["Temperature"] = df["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)
    
    return df   

In [737]:
answer["Voltage range"]

0       2.5-4.3
1       2.5-4.3
2       2.5-4.3
3       2.5-4.3
4     2.50-4.50
5     2.50-4.50
6       3.0-4.3
7       3.0-4.3
8       3.0-4.3
9       3.0-4.3
10      3.0-4.3
11      3.0-4.3
12      3.0-4.3
13      2.8-4.3
14      2.8-4.3
15    not found
16      3.0?4.3
17      2.6-4.2
18      2.6-4.2
19      2.6-4.2
20    2.75?4.30
21    2.75?4.30
22    not found
23     3.0-4.55
24      3.0-4.3
25      3.0-4.3
26          NaN
27          NaN
Name: Voltage range, dtype: object

In [738]:
gt = text_processor(gt)
answer = text_processor(answer)

In [739]:
Morphological_Properties_col = ['Particle size', 'Particle shape', 'Particle distribution', 'Coating layer characteristics', 'Crystal structure and lattice characteristics']

In [740]:
gt = gt.drop(columns=Morphological_Properties_col)
answer = answer.drop(columns=Morphological_Properties_col)

In [741]:
def c_rate_func(x):
    if "C-rate" in x:
        return True
    else: 
        return False

In [742]:
def combine_duplicated_cols(df):
    int_crate_cols = [
        col for col in df.columns
        if re.fullmatch(r"C-rate\s+[0-9]+", col)
    ]    
    
    int2float_map = {}
    for int_col in int_crate_cols:
        int2float_map[f"{int_col}"] = int_col[:7] + f"{float(int_col[7:])}"

    for int_col, float_col in int2float_map.items():
        if float_col in df.columns:
            df[int_col] = df[int_col].combine_first(df[float_col])
            df.drop(columns=float_col, inplace=True)
        df.rename(columns={int_col: float_col}, inplace=True)
    
    return df

In [743]:
gt = combine_duplicated_cols(gt)
answer = combine_duplicated_cols(answer)

In [744]:
gt_c_rate = gt.columns[gt.columns.map(c_rate_func)].to_list()
answer_c_rate = answer.columns[answer.columns.map(c_rate_func)].to_list()

In [745]:
# other_c_rate_col = [
#     'C-rate 0.05',
#     'C-rate 5.0',
#     'C-rate 10.0',
#     'C-rate 20.0',
#     'C-rate 40.0',
#     'C-rate 0.33',
#     'C-rate None',
#     'C-rate 0.2',
#     'C-rate 0.3',
#     'C-rate 4.0',
#     'C-rate 6.0',
# ]

In [746]:
paper_id = gt["Paper ID"].unique()

In [747]:
gt.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0,Voltage range min,Voltage range max
0,11,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,45.5,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
1,11,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
2,11,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,75.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
3,11,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,95.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
4,16,NCM,1,0.33,0.33,0.33,2,NaN,no,LiAc,...,143.5,101.7,71.5,44.5,13.2,NaN,NaN,NaN,2.5,4.5


In [748]:
answer.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0,Voltage range min,Voltage range max
0,11.0,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,45.5,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
1,11.0,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,47,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
2,11.0,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,75.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
3,11.0,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,95.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
4,16.0,NCM,1,0.33,0.33,0.33,NaN,NaN,no,LiAc,...,143.5,101.7,71.5,44.5,13.2,NaN,NaN,NaN,2.5,4.5


In [749]:
answer.columns

Index(['Paper ID', 'Sample', 'Li ratio', 'Ni ratio', 'Co ratio', 'Mn ratio',
       'O ratio', 'W ratio', 'Commercial NCM used', 'Lithium source',
       'Synthesis method', 'Crystallization method',
       'Crystallization final temperature',
       'Crystallization final duration (hours)', 'Doping', 'Coating',
       'Active material to Conductive additive to Binder ratio',
       'Electrolyte salt', 'Electrolyte concentration', 'Electrolyte solvent',
       'Electrolyte solvent ratio', 'Additive',
       'Loading density (mass loading of NCM)', 'Temperature', 'C-rate 0.05',
       'C-rate 0.1', 'C-rate 0.2', 'C-rate 0.5', 'C-rate 1.0', 'C-rate 2.0',
       'C-rate 5.0', 'C-rate 10.0', 'C-rate 20.0', 'C-rate 40.0',
       'C-rate None', 'C-rate 4.0', 'C-rate 6.0', 'Voltage range min',
       'Voltage range max'],
      dtype='object')

In [750]:
sort_col = [
    'Paper ID',
    'Sample',
    'Li ratio',
    'Ni ratio',
    'Co ratio',
    'Mn ratio',
    'O ratio',
    'W ratio',
    'Commercial NCM used',
    'Lithium source',
    'Synthesis method',
    'Crystallization method',
    'Crystallization final temperature',
    'Crystallization final duration (hours)',
    'Doping',
    'Coating',
    'Active material to Conductive additive to Binder ratio',
    'Electrolyte salt',
    'Electrolyte concentration',
    'Electrolyte solvent',
    'Electrolyte solvent ratio',
    'Additive',
    'Loading density (mass loading of NCM)',
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.05',
    'C-rate 0.1', 
    'C-rate 0.2', 
    'C-rate 0.5', 
    'C-rate 1.0', 
    'C-rate 2.0',
    'C-rate 5.0', 
    'C-rate 10.0', 
    'C-rate 20.0', 
    'C-rate 40.0',
    'C-rate None', 
    'C-rate 4.0', 
    'C-rate 6.0'
]

In [751]:
tf_col = [
    "Li ratio", 
    "Ni ratio", 
    "Co ratio", 
    "Mn ratio", 
    "O ratio", 
    "W ratio", 
    "Commercial NCM used",
    'Lithium source',
    "Crystallization final temperature", 
    "Crystallization final duration (hours)", 
    "Electrolyte salt", 
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.05',
    'C-rate 0.1', 
    'C-rate 0.2', 
    'C-rate 0.5', 
    'C-rate 1.0', 
    'C-rate 2.0',
    'C-rate 5.0', 
    'C-rate 10.0', 
    'C-rate 20.0', 
    'C-rate 40.0',
    'C-rate None', 
    'C-rate 4.0', 
    'C-rate 6.0'
]

In [752]:
import pandas as pd

def is_number(val):
    try:
        float(val)
        return True
    except:
        return False

# key column 정의
key_cols = ["Paper ID", "Sample"]

# 모든 열 모음 (비교 대상)
all_cols = set(gt.columns) | set(answer.columns)
compare_cols = [col for col in all_cols if col not in key_cols]

# 결과 저장
results = []

# 모든 논문 처리
all_papers = sorted(set(gt["Paper ID"]) | set(answer["Paper ID"]))

for pid in all_papers:
    gt_group = gt[gt["Paper ID"] == pid].reset_index(drop=True)
    ans_group = answer[answer["Paper ID"] == pid].reset_index(drop=True)

    max_len = max(len(gt_group), len(ans_group))

    def pad_df(df, target_len):
        pad_len = target_len - len(df)
        if pad_len <= 0:
            return df
        empty = pd.DataFrame({col: [None] * pad_len for col in df.columns})
        empty["Paper ID"] = pid
        return pd.concat([df, empty], ignore_index=True)

    gt_group = pad_df(gt_group, max_len)
    ans_group = pad_df(ans_group, max_len)

    for i in range(max_len):
        row_gt = gt_group.iloc[i]
        row_ans = ans_group.iloc[i]

        result_row = {
            "Paper ID": pid,
            "Sample": row_gt["Sample"] if pd.notna(row_gt["Sample"]) else row_ans["Sample"]
        }

        for col in compare_cols:
            val_gt = row_gt[col] if col in row_gt and col in gt.columns else None
            val_ans = row_ans[col] if col in row_ans and col in answer.columns else None

            try:
                # C-rate 열이고 둘 다 NaN이면, 결과도 NaN
                if "C-rate" in col and pd.isna(val_gt) and pd.isna(val_ans):
                    match = float('nan')
                elif pd.isna(val_gt) and pd.isna(val_ans):
                    match = True
                elif pd.notna(val_gt) and pd.notna(val_ans):
                    if is_number(val_gt) and is_number(val_ans):
                        match = abs(float(val_gt) - float(val_ans)) <= 0.001
                    else:
                        match = str(val_gt).strip() == str(val_ans).strip()

                else:
                    match = False

            except:
                match = False

            # 결과 저장 방식 변경
            if col in tf_col:
                result_row[col] = match
            else:
                result_row[col] = val_ans  # 숫자 컬럼이 아닌 경우는 answer 값 그대로 저장

        results.append(result_row)

# 결과 DataFrame
df_result = pd.DataFrame(results)

# 누락된 열 채우기 (혹시라도)
for col in tf_col:
    if col not in df_result.columns:
        df_result[col] = False

# 컬럼 순서 정렬
df_result = df_result[sort_col]

# 저장
df_result.to_csv(f"../output/csv/acc_{hyper_method_name}.csv", index=False)

In [753]:
answer

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0,Voltage range min,Voltage range max
0,11.0,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,45.5,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
1,11.0,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,47,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
2,11.0,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,75.4,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
3,11.0,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,95.4,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
4,16.0,NCM,1,0.33,0.33,0.33,NaN,NaN,no,LiAc,...,143.5,101.7,71.5,44.5,13.2,NaN,NaN,NaN,2.50,4.50
5,16.0,NCM/C,1,0.33,0.33,0.33,NaN,NaN,no,LiAc,...,163.5,135.8,109.4,81.9,54.6,NaN,NaN,NaN,2.50,4.50
6,22.0,Pristine,1,0.84,0.1,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
7,22.0,V-0.005,1,0.84,0.1,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
8,22.0,V-0.01,1,0.84,0.1,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
9,22.0,V-0.02,1,0.84,0.1,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30


In [754]:
gt

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0,Voltage range min,Voltage range max
0,11,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,45.50,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
1,11,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,47.00,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
2,11,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,75.40,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
3,11,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,95.40,NaN,NaN,NaN,NaN,NaN,NaN,2.50,4.30
4,16,NCM,1,0.33,0.33,0.33,2,NaN,no,LiAc,...,143.5,101.70,71.5,44.5,13.2,NaN,NaN,NaN,2.50,4.50
5,16,NCM/C,1,0.33,0.33,0.33,2,NaN,no,LiAc,...,163.5,135.80,109.4,81.9,54.6,NaN,NaN,NaN,2.50,4.50
6,22,Pristine,1,0.84,0.10,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
7,22,V-0.005,1,0.84,0.10,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
8,22,V-0.01,1,0.84,0.10,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,4.30
9,22,V-0.02,1,0.84,0.10,0.06,2,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [755]:
df_result

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 0.5,C-rate 1.0,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0
0,11,NR0,True,True,True,True,True,True,True,True,...,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,NaN,NaN,NaN
5,16,NCM/C,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,NaN,NaN,NaN
6,22,Pristine,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,22,V-0.005,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22,V-0.01,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,22,V-0.02,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [756]:
answer[["Sample", "Lithium source", 'Voltage range min', 'Voltage range max', 'Temperature']]

,Sample,Lithium source,Voltage range min,Voltage range max,Temperature
0,NR0,LiNO3,2.50,4.30,NaN
1,NR1,LiNO3,2.50,4.30,NaN
2,NR3,LiNO3,2.50,4.30,NaN
3,NR5,LiNO3,2.50,4.30,NaN
4,NCM,LiAc,2.50,4.50,30
5,NCM/C,LiAc,2.50,4.50,30
6,Pristine,LiOH,3.00,4.30,25
7,V-0.005,LiOH,3.00,4.30,25
8,V-0.01,LiOH,3.00,4.30,25
9,V-0.02,LiOH,3.00,4.30,25


In [757]:
df_result[["Sample", "Lithium source", 'Voltage range min', 'Voltage range max', 'Temperature']]

,Sample,Lithium source,Voltage range min,Voltage range max,Temperature
0,NR0,True,True,True,False
1,NR1,True,True,True,False
2,NR3,True,True,True,False
3,NR5,True,True,True,False
4,NCM,True,True,True,True
5,NCM/C,True,True,True,True
6,Pristine,True,True,True,True
7,V-0.005,True,True,True,True
8,V-0.01,True,True,True,True
9,V-0.02,True,False,False,False
